# 3D Rigid Body Motion

## Using Numpy/Scipy as a replacement for Eigen

In [66]:
import numpy as np

### Basic Matrix Ops

In [67]:
m = np.random.randn(3,3)
print(f"Transpose = {m.T}")
print(f"\nSum: {m.sum()}")
print(f"Trace: {m.trace()}")
print(f"\nScalar multiplication: {m * 10}")
print(f"\nInverse matrix: {np.linalg.inv(m)}") # <- Computed inverse using LU factorisation
print(f"\nDeterminant: {np.linalg.det(m)}")

Transpose = [[ 0.09901086 -1.11990578  0.1665566 ]
 [-1.44058641 -0.6945656  -1.27538648]
 [-1.72246644  0.72816944 -0.04993891]]

Sum: -5.309112727511681
Trace: -0.6454936449429404

Scalar multiplication: [[  0.99010865 -14.40586412 -17.2246644 ]
 [-11.19905785  -6.94565598   7.28169437]
 [  1.66556598 -12.7538648   -0.49938912]]

Inverse matrix: [[-0.36241304 -0.79934982  0.84467588]
 [-0.02458558 -0.10606376 -0.69854442]
 [-0.5808329   0.04275828  0.63278196]]

Determinant: -2.6582467838983166


In [68]:
# Eigen values
s = m.T @ m # construct a real symmetric matrix: A^T A
eigVals, eigVecs = np.linalg.eigh(s) # `eigh` is optimised for symmetric matrices, compared to the generalist `eig`
print(f"Eigen Values: {eigVals}")
print(f"Eigen Vectors: {eigVecs}")

Eigen Values: [0.46141632 2.58117607 5.93307734]
Eigen Vectors: [[ 0.78526704  0.61487933 -0.07265731]
 [-0.36366628  0.55301889  0.7496112 ]
 [ 0.50110129 -0.56222195  0.65787839]]


In [69]:
# Solving Equations
sz = 100
A = np.random.randn(sz, sz)
A = A @ A.T # Make `A` into a positive semi-definite matrix (x^T.M.x >= 0) by representing it as a symmetric matrix
b = np.random.randn(sz)
# By positive semi-definiteness, `b.T @ A @ b` >= 0

In [70]:
# by direct inversion using LU factorisation
x = np.linalg.inv(A) * b
print(f"Condition number for A: {np.linalg.cond(A)}, (could be worse)")
print(np.linalg.norm(x))

Condition number for A: 76999.02863004227, (could be worse)
175.88641413984547


In [71]:
# QR decomposition
import scipy
def qr_solve(A, b):
    Q, R = np.linalg.qr(A)
    # Q is orthonormal, therefore, its transpose is also its inverse
    p = Q.T @ b
    Rinv = np.linalg.inv(R) # can't be bothered to solve directly using upper triangular R structure
    return Rinv @ p
    # return scipy.linalg.solve_triangular(R, p)

x = qr_solve(A, b)
print(np.linalg.norm(x)) # <- norm is much lower here, QR decomp is much more stable and accurate

269.4752292531543


In [72]:
# Cholesky decomp
L = np.linalg.cholesky(A)
y = np.linalg.solve(L, b)
x = np.linalg.solve(L.T, y)
print(np.linalg.norm(x))

269.4752292528302
